# 循环神经网络RNN

RNN是一种主要用于处理时序数据的深度模型. 其常涉及的工作如:

- **语音识别(Speech recognition)**: 给一段讲话的语音,将其转化为文字表述(多对多);
- **音乐生成(Music generation)**: 给一个音符, 生成一段音乐(一对多);
- **情感分类(Sentiment classification)**: 给一段某商品下用户的留言, 为用户留言的情感分类(消极或者积极)(多对一);
- **机器翻译(Machine translation)**: 将一种语言翻译成另一种语言(多对多);
- **视频运动分析(Video activity recognition)**: 给一段视频,然后要求你识别其中的行为(多对一);
- **命名实体识别(Name entity recognition)**: 给定一个句子要你识别出句中的人名(多对多);

可以发现,上面问题的输入$X$输出$Y$大部分都是**时序序列**, 这是RNN可处理问题的一大特征.

下面将从命名实体识别问题探讨如何从0开始建立一个RNN网络。

## 命名实体识别（Name entity recognition）

模型想处理的问题是找出一句话中的人名，比如输入$X$是：**"Harry Potter and Herminoe Granger invented a new spell."** (这些人名都是出自于J.K.Rowling笔下的系列小说Harry Potter)，输出$Y$也是一个序列，其表示了每一个单词是否是人名的一部分。我们用下面的图表示模型：

![rnn1](./resources/rnn1.png)

- 我们使用$x^{<1>}, x^{<2>}, ..., x^{<t>},..., x^{<9>}$表示输入的每一个单词，他们是一个**时序序列**，会按照时间先后顺序输入模型。另一方面，使用$T_x=9$表示输入序列的时间长度。
- 同样$y^{<1>}, y^{<2>}, ..., y^{<t>},..., y^{<9>}$表示每一个时刻模型的输出。使用$T_y=9$表示输出序列的时间长度。
- 针对多组训练数据，使用$x^{(i)<t>}$表示第$i$组训练数据的第$t$时刻的输入；使用$y^{(i)<t>}$表示第$i$组训练数据的第$t$时刻的输出；使用$T_x^{(i)}$表示第$i$组训练数据输入序列的时间长度；使用$T_y^{(i)}$表示第$i$组训练数据输出序列的时间长度。

### 如何表征输入？

上面模型每一个时刻的输入是单词，单词不能直接输入模型，需要转换成某种统一的表示后网络才能处理。最常用的表征方式如**one-hot方式**。one-hot即先排列出数据集中所有出现的词，然后表征每一个词用该位置1，其他位置0，这样生成的一个稀疏的one-hot向量来表示。

注意：one-hot有两个缺点：太稀疏（浪费空间）；丢失词之间的关系。

### 使用标准神经网络

针对上面的命名实体识别，可以尝试的方法之一就是使用**标准神经网络**，其建立的网络结构可如下：

![rnn2](./resources/rnn2.png)

但是结果表明这个方法并不好，主要存在的问题可能在于：

- **输入序列和输出序列在不同例子中可以有不同的长度**，即使可以设置最大长度填充，但仍然不是较好的表示方法。
- **不共享从文本的不同位置上学到的特征**，即句子前面部分学习到的信息不能在后面部分使用。比如前面已经学习到**Harry**是人名的一部分，句子后面部分又出现了这个词可能还是无法被识别出来。
- **参数巨量**，使用One-hot表示的句子作为上面网络的输入，第一层会有海量的参数。

### 循环神经网络

下面介绍另一种处理这种时序数据的模型：循环神经网络。它的结构如下：

![rnn3](./resources/rnn3.png)

从左往右依次读入句中每一个单词，将第一个单词$x^{<1>}$输入网络，然后尝试预测输出$y^{<1>}$，接着输入$x^{<2>}$来预测$y^{<2>}$，但不一样的地方在于，预测$y^{<2>}$的时候**会使用前一时刻产生的一个激活值$a^{<1>}$作为输入**，这样顺序进行直到输入结束。

需要注意的时候，上面1图是将RNN在时间上展开了，真正的结构更像2图所示，一个RNN是共用**一种结构和一套权值（$W_{ax}, W_{aa}, W_{ya}$）**，且一般我们构造$a^{<0>}$为全0向量。

另一点需要注意的是，上面神经网络结构的一个限制是它在某一时刻的预测**仅使用了序列之前的输入信息而并没有使用序列中后部分的信息**，后面介绍的双向RNN可以改进这一点。

假设我们将第一个时刻的RNN内部结构展开，可得到如下结构图：

<img src="./resources/rnn4.jpg" style="width:60%">

按照上图所示，可得到如下计算式子：

> $a^{<1>}=g_1(W_{aa}a^{<0>}+W_{ax}x^{<1>}+b_a)$  
> $\hat{y}^{<1>}=g_2(W_{ya}a^{<1>}+b_y)$

一般$g_1$常使用**tanh**或**ReLU**，$g_2$则取决于输出，如二分类就可能用sigmoid，多分类可能用softmax。**要注意的是上面的$W_{ax}, W_{aa}, W_{ya}, b_a, b_y$是整个网络公用的一套参数，它在每一个时刻都被使用，需要训练的也是这些参数**。

<img src="./resources/rnn8.png">


可将上面的符号简化，下面列出了简化后的符号表示：

> $a^{<1>}=g_1(W_{a}[a^{<0>},x^{<1>}]+b_a)$  
> $\hat{y}^{<1>}=g_2(W_{y}a^{<1>}+b_y)$

假设$x^{<i>}$的维度为500, $a^{<i-1>}$的维度都为100，$y^{<i>}$维度为5，则共有4组参数：

- 隐藏层维度为100（等于$a^{<i-1>}$的维度）；
- $W_a$的维度为$(500+100, 100)$，$b_a$等于$(100, )$；
- $W_y$维度等于$(100, 5)$,$b_y$的维度为$(5, )$；

### 时间反向传播

下面介绍RNN的反向传播算法，算法流程如下：

![rnn5](./resources/rnn5.png)

上图蓝色箭头即为前向传播（Foward propagation）过程，每一个时刻产生一个预测值$\hat{y}^{<t>}$，然后跟标签值$y^{<t>}$一起计算这个时刻的误差：

> $L^{<t>}(\hat{y}^{<t>}, y^{<t>})=-y^{<t>}log\hat{y}^{<t>}-(1-y^{<t>})log(1-\hat{y}^{<t>})$

我们会将整个序列的误差累计起来：

> $L(\hat{y}, y)=\sum_{t=1}^{T_x}L^{<t>}(\hat{y}^{<t>}, {y}^{<t>})$

反向传播（Backpropagation）的过程即**从4向下向左反向传播误差（红色箭头）**。

### 不同类型的RNN

上面命名实体识别是一种多对多（many-to-many）的结构，其实还有其他的：

- 一对一（one-to-one）：其实就是标准神经网络；
- 一对多（one-to-many）：如音乐生成（& -> 12345...）；
- 多对一（many-to-one）：如情感分类（'These is nothing to like in this movie' -> 1分）；
- 多对多（many-to-many）：考虑输入和输出长度不同的情况，如机器翻译；

下面展示了各种结构的图示：

![](./resources/rnn6.png)

### 语言模型（Language model）

构建语言模型是NLP种的最重要问题之一。语言模型的功能就是：**构建一个模型可以预测一句话发生的可能性**。一个应用场景：假设我们有一个语音识别系统，一个人讲了一句话，系统给了两句可能的输出，这个时候就可以用语言模型来判断每句话发生的可能，选择可能性较大的作为输出。

语言模型会估计某个句子序列中各个单词出现的可能性，再将它们的可能性乘起来获得句子的可能性。

下面介绍如何训练一个语言模型：

首先，需要一个很大的英语语料库，其里面有很多句子，我们就用这些句子不停的去训练语言模型，让**语言模型尽可能生成符合句子在语料库中频率分布的可能性输出**（如一个句子出现很多次，其使用语言模型生成的概率也应该很高）。下面是训练的细节：

![](./resources/rnn7.png)

比如现在有这么一句话："Cats average 15 hours of sleep a day."。在0时刻，设置$a^{<0>}=\vec{0}, \ x^{<1>}=\vec{0}$，输出为词库中所有词可能是第一个词的概率，对于这一句话来讲，我们希望$p(Cats)$比较大；接着再给网络输入：$x^{<2>}=\vec{Cats}$，即算在第一个词是Cats的情况下，第二个词的分布概率，后面也继续这样操作。最后根据预测分布于实际Label产生每一个时刻的误差，再使用这个误差来修正网络。

需要注意：训练的目的在于希望让模型生成被训练这一句话的概率会大一点。这样多次训练，模型生成的句子的概率会越来越接近它们在语料库中出现的频率。

这里可能就产生了另一个语言模型的应用：给定前面几个词，预测剩余词的任务。或者我们可以直接用这个模型来生成一些符合语料库风格的序列（新序列采样任务）。

### RNN梯度消失

考虑这样一个场景："The cat, which ..., is black."，因为cat是单数，所以使用is，即is与cat之间有一种长期依赖。但是实际上RNN并不擅长捕捉这种太长的关联信息，原因如下：

假设RNN很长（时序上），先前馈传播，再反向传播。但是因为太深，后面产生的梯度会很难传到前面去，所以它能记住的时序依赖关系的间隔不太长。

梯度难以传回的原因：RNN的输出会作为下一层的输入，类似复合函数，那么根据链式求导法则，求导时各个函数的导数要求乘积，假设如果导数很多小于1，就会让梯度消失。

**梯度消失是训练RNN的首要的问题**，梯度爆炸也会出现，但可以使用修剪解决。

下面介绍的GRU和LSTM可以有效的解决梯度消失的问题。

### 长短时记忆LSTM

LSTM于1997年被提出，其功能非常强大，整体示意图可见如下：

![](./resources/rnn9.jpg)

其在时序间传递的信息不止隐藏状态$h_t$（底部的直线），还包括了cell记忆值$C_t$（贯穿顶部的直线），并通过引入遗忘门，输入门，输出门等来控制隐藏状态与记忆的更新。门可以描述通过的比例（0意味着什么都不能通过，1意味着让什么都通过）。

#### 1.遗忘门（forget gate）

$f_t$表示遗忘门，其决定了从cell中扔掉什么信息，其操作图示如下：

![](./resources/rnn10.jpg)

#### 2.输入门（input gate）

首先我们计算出候选cell值，再使用输出门$i_t$来决定我们更新候选值到cell值的幅度，其图示如下：

![](./resources/rnn11.jpg)

#### 3.更新

根据遗忘门与输出门对Cell值进行更新得到$C_t$：

![](./resources/rnn12.jpg)

#### 4.输出门（output gate）

我们还使用一个输出门$o_t$决定输出哪些cell的值到新的隐藏状态$h_t$：

![](./resources/rnn13.jpg)

LSTM解决梯度消失：状态S是累加方式，导数不再是乘积形式（还需确认意思）。

为了分析如LSTM单元中的内部操作，我们举一个例子，比如一批训练共64句话，每句话20个单词，每个词向量长度为200，隐藏层单元个数为128。那么训练一批句子，输入的张量维度是`[64, 20, 200]`，$h_t$与$c_t$的维度都是`[128,]`，LSTM参数矩阵维度为`[128+200, 4*128]`（乘以4是因为LSTM有四个位置都要计算输出）。

在第一个时刻，将64句话的第一个单词作为输入，即输入为`[64, 200]`，然后需要与$h_t$拼接，输入变为了`[64, 200+128]`，然后与参数矩阵相乘得到`[64, 4*128]`，就是每个需要计算输出的地方都是`[64, 128]`，这样做完20个时刻，总的输出为`[20, 64, 128]`。

### 门控循环单元GRU

于2014年被提出，其组合遗忘门与输出门到更新门，并合并了cell state与hidden state，模型相比于LSTM更简单，适合构建深度网络：

![](./resources/rnn14.jpg)

包含两个门：重置门（reset gate）和更新门（update gate）。

## 参考文献

- [TensorFlow实战：LSTM的结构与cell中的参数](https://blog.csdn.net/wjc1182511338/article/details/79689409)